# TODO: add and explain plots, explain experiments, 


# Active RL experiment progress report

We're looking at Reinforcement learning in Tabular MDPs.
In our setting, rewards are only observed when the agent makes a decision to query them (paying some cost to do so).
In order to maximize performance (measured as returns minus total query cost), the agent must intelligently chose when to make these queries.

So far, we focus on a simple heuristic which queries any state-action which has not already been queried N times, where N is a hyper-parameter.  

We propose two ways of tuning this hyperparameter and demonstrate their potential in preliminary experiments. 
Note that tuning N by running multiple experiments in the environment is assumed not to be an option in our motivating set-up, since this would impose real costs (both in terms of queries, and, potentially, rewards).

## Environment
We've been experimenting with gridworld and chain environments. 
The chain environment is used by Ian Osband in this paper:
https://arxiv.org/abs/1607.00215

In this environment, the agent must explore a long chain of states, only the last of which has positive expected reward.

We'll present results for the chain of length 5.
Following Osband, we use PSRL in all of our experiments.

Our code is available in a fork of Ian's repo: 
https://github.com/capybaralet/TabulaRL

In [10]:
# Here is the implementation of the chain environment:

from TabulaRL.environment import TabularMDP
def make_stochasticChain(chainLen, max_reward=1):
    nState = chainLen
    epLen = chainLen
    nAction = 2
    pNoise = 1. / chainLen

    R_true = {}
    P_true = {}
    for s in xrange(nState):
        for a in xrange(nAction):
            R_true[s, a] = (0, 0)
            P_true[s, a] = np.zeros(nState)

    # Rewards (the start and end state rewards are stochastic, all others are determinstically 0.)
    R_true[0, 0] = (0, 1)
    R_true[nState - 1, 1] = (max_reward, 1)

    # Transitions 
    for s in xrange(nState):
        P_true[s, 0][max(0, s-1)] = 1.

        # the forward action doesn't always succeed:
        P_true[s, 1][min(nState - 1, s + 1)] = 1. - pNoise
        P_true[s, 1][max(0, s-1)] += pNoise

    stochasticChain = TabularMDP(nState, nAction, epLen)
    stochasticChain.R = R_true
    stochasticChain.P = P_true
    stochasticChain.reset()
    return stochasticChain


## Our experiments

Our experiments so far aim at testing the validity of a few of our ideas (and their implementations).



# Idea 1: 
Since the reason PSRL samples environments is to encourage exploration, and we may stop querying (state, action) pairs at some point, we propose using the expected reward (instead of a sample) for any such (s,a).  Our first experiment verifies that this improves performance.

# Idea 2: 

In order to tune the value of N, we propose evaluating different values of N in environments sampled from the agent's posterior, and chosing the N with the best average performance in these experiments; we call this algorithm "simulated query rollouts (SQR)".  

We consider an approximate version of SQR ("ASQR"). Instead of actually running experiments in the simulated environments (which can be prohibitively costly), ASQR instead:
1. Samples an environment, E, from the agents posterior
2. Performs all N queries of each (s,a) off-line without acting in E
3. Updates the agent's posterior based on the results of these queries
4. Finds the expected environment, E', based on this updated posterior
5. Uses planning to find the optimal policy for E', and computes the expected return of running this policy in E
6. Selects the N with the best performance (= expected return - cost of queries).

We evaluate both of these algorithms as well, and show that their estimates of the best N do not deviate drastically from the best N found by hyper-parameter search.  


For a more thorough description of SQR and ASQR, see: https://github.com/capybaralet/TabulaRL/blob/master/tex/Choosing%20Which%20Queries%20to%20Make.pdf





# TODO: more comments in the code
Here, we include a modified version of the script we use to run these experiments (dk_exp_script.py), removing parts which don't work in an ipython notebook (like argparser). 

In [9]:
# %load dk_exp_script.py
import numpy as np
import argparse
from collections import defaultdict

import TabulaRL.gridworld as gridworld
import TabulaRL.query_functions as query_functions
import TabulaRL.finite_tabular_agents as finite_tabular_agents
from TabulaRL.feature_extractor import FeatureTrueState
from TabulaRL.environment import make_stochasticChain


#-----------------------------------------------------------------------------------
# USEFUL FUNCTIONS

def is_power2(num):
    'states if a number is a power of two'
    return num != 0 and ((num & (num - 1)) == 0)

def sample_gaussian(loc, scale, shape):
    if scale == 0:
        return loc * np.ones(shape)
    else:
        return np.random.normal(loc, scale, shape)

#-----------------------------------------------------------------------------------
# SETUP
algorithm='fixed_n'  
algorithm='SQR'  
algorithm='ASQR'  

log_n_max=10
log_num_episodes=10
num_R_samples=20
query_cost = 1.
normalize_rewards=0

save_str = 'toy_experiment/'

# ENVIRONMENT
chain_len = 5
epLen = chain_len
env = make_stochasticChain(chain_len, max_reward=((chain_len - 1.)/chain_len)**-chain_len)
f_ext = FeatureTrueState(env.epLen, env.nState, env.nAction, env.nState)

# AGENT
alg = finite_tabular_agents.PSRLLimitedQuery

# RUN EXPERIMENT
initial_env = env
num_episodes_remaining = 2**log_num_episodes
n_max = 2**log_n_max
ns = np.hstack((np.array([0,]), 2**np.arange(log_n_max)))

# save results here:
num_queries = np.empty((num_R_samples, log_num_episodes+1, log_n_max+1))
returns = np.empty((num_R_samples, log_num_episodes+1, log_n_max+1))
returns_max_min = np.empty((num_R_samples, 2))


for kk in range(num_R_samples):
    print "beginning experiment #", kk
    env = copy.deepcopy(initial_env)

    if algorithm in ['SQR', 'ASQR']: # use a sampled environment
        sampled_R, sampled_P = initial_agent.sample_mdp()
        env.R = {kk:(sampled_R[kk], 1) for kk in sampled_R}
        env.P = sampled_P
        returns_max_min[kk,0] = initial_agent.compute_qVals(sampled_R, sampled_P)[1][0][0]
        returns_max_min[kk,1] = - initial_agent.compute_qVals({kk: -sampled_R[kk] for kk in sampled_R}, sampled_P)[1][0][0]

    sampled_rewards = {(s,a) : sample_gaussian(env.R[s,a][0], env.R[s,a][1], n_max) for (s,a) in env.R.keys()}
    # is this still needed??
    first_n_sampled_rewards = [{sa: sampled_rewards[sa][:n] for sa in sampled_rewards} for n in range(n_max + 1)]
    for ind, n in enumerate(ns):
        agent = copy.deepcopy(initial_agent)
        if environment.startswith('grid'): # FIXME: update agent
            query_function = QueryFixedFunction(query_cost, lambda s,a: (a==0) * n)
        else:
            query_function = query_functions.QueryFirstNVisits(query_cost, n)
        query_function.setAgent(agent)

        if algorithm=="ASQR": # update posterior, compute expected returns
            updated_R = {}
            for [s,a] in first_n_sampled_rewards[n]:
                mu0, tau0 = agent.R_prior[s,a]
                num_samples = len(first_n_sampled_rewards[n][s,a])
                tau1 = tau0 + agent.tau * num_samples
                mu1 = (mu0 * tau0 + sum(first_n_sampled_rewards[n][s,a]) * agent.tau) / tau1
                updated_R[s,a] = mu1
            updated_P = sampled_P
            expected_returns = agent.compute_qVals_true(updated_R, updated_P, sampled_R, sampled_P)[0]
            returns[kk, :, ind] = expected_returns * 2**np.arange(log_num_episodes+1)
            num_queries[kk, :, ind] = n * sum([agent.query_function.will_query(s,a) for [s,a] in first_n_sampled_rewards[n]])
        else: # Run an experiment 
            nEps = num_episodes_remaining
            # --------------- modified from dk_run_finite_tabular_experiment ------------------
            qVals, qMax = env.compute_qVals()
            cumReward = 0
            cumQueryCost = 0 
            for ep in xrange(1, nEps + 2):
                env.reset()
                epMaxVal = qMax[env.timestep][env.state]
                agent.update_policy(ep)

                pContinue = 1
                while pContinue > 0:
                    # Step through the episode
                    h, oldState = f_ext.get_feat(env)

                    action = agent.pick_action(oldState, h)
                    query, queryCost = agent.query_function(oldState, action, ep, h)
                    cumQueryCost += queryCost

                    reward, newState, pContinue = env.advance(action)
                    if query and first_n_sampled_rewards[n] is not None:
                        reward = first_n_sampled_rewards[n][oldState, action][agent.query_function.visit_count[oldState, action] - 1]
                    cumReward += reward 
                    agent.update_obs(oldState, action, reward, newState, pContinue, h, query)

                if is_power2(ep): # checkpoint
                    returns[kk, int(np.log2(ep)), ind] = cumReward
                    num_queries[kk, int(np.log2(ep)), ind] = cumQueryCost / query_cost

            # ---------------------------------------------------------------------
    np.save(save_str + 'num_queries', num_queries)
    np.save(save_str + 'returns', returns)
    np.save(save_str + 'returns_max_min', returns_max_min)




ArgumentError: argument --log_n_max: conflicting option string(s): --log_n_max

# Analysis (TODO: rm?)
The way we've implemented the above experiments, we can use our saved results to compute the performance for many different query costs and numbers of episodes.

So, for instance, if we want to see how performance changes as a function of 